In [4]:
import datetime
import itertools
import concurrent.futures
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random
from scipy.optimize import minimize
from scipy import linalg
import seaborn as sns
import time

# # organize DataSeries
# date = data.index
# tLen = len(data)
# time = np.linspace(0, tLen-1, tLen)
# close = [data["Adj Close"][i] for i in range(len(data["Adj Close"]))]
# DataSeries = [time, close]

# matrix helpers
def _yi(price_series):
    return [np.log(p) for p in price_series]

def _fi(tc, m, time_series):
    return [np.power((t - tc), m) for t in time_series]

def _gi(tc, m, w, time_series):
    return [np.power((t - tc), m) * np.cos(w * np.log(t - tc)) for t in time_series]

def _hi(tc, m, w, time_series):
    return [np.power((t - tc), m) * np.sin(w * np.log(t - tc)) for t in time_series]

def _fi_pow_2(tc, m, time_series):
    return np.power(_fi(tc, m, time_series), 2)

def _gi_pow_2(tc, m, w, time_series):
    return np.power(_gi(tc, m, w, time_series), 2)

def _hi_pow_2(tc, m, w, time_series):
    return np.power(_hi(tc, m, w, time_series), 2)

def _figi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _gi(tc, m, w, time_series))

def _fihi(tc, m, w, time_series):
    return np.multiply(_fi(tc, m, time_series), _hi(tc, m, w, time_series))

def _gihi(tc, m, w, time_series):
    return np.multiply(_gi(tc, m, w, time_series), _hi(tc, m, w, time_series))

def _yifi(tc, m, time_series, price_series):
    return np.multiply(_yi(price_series), _fi(tc, m, time_series))

def _yigi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _gi(tc, m, w, time_series))

def _yihi(tc, m, w, time_series, price_series):
    return np.multiply(_yi(price_series), _hi(tc, m, w, time_series))

In [5]:
# revised version of the LPPL without φ
# found on page 11 as equation (13)
def lppl(t, tc, m, w, a, b, c1, c2):
    return a + np.power(t - tc, m) * (b + ((c1 * np.cos(w * np.log(t - tc))) + (c2 * np.sin(w * np.log(t - tc)))))

# finds the least square difference
def func_restricted(x, *args):
    tc = x[0]
    m  = x[1]
    w  = x[2]
    
    data_series = args[0]
    
    lin_vals = matrix_equation(tc, m, w, data_series)
    
    a  = lin_vals[0] 
    b  = lin_vals[1]
    c1 = lin_vals[2] 
    c2 = lin_vals[3]
    
    delta = [lppl(t, tc, m, w, a, b, c1, c2) for t in data_series[0]]
    delta = np.subtract(delta, data_series[1])
    delta = np.power(delta, 2)
    return np.sum(delta)


# solve the matrix equation
def matrix_equation(tc, m, w, data_series):
    time_series = data_series[0]
    price_series = data_series[1]
    N  = len(price_series)
    
    fi = np.sum(_fi(tc, m, time_series))
    gi = np.sum(_gi(tc, m, w, time_series))
    hi = np.sum(_hi(tc, m, w, time_series))
    
    fi_pow_2 = np.sum(_fi_pow_2(tc, m, time_series))
    gi_pow_2 = np.sum(_gi_pow_2(tc, m, w, time_series))
    hi_pow_2= np.sum(_hi_pow_2(tc, m, w, time_series))
    
    figi = np.sum(_figi(tc, m, w, time_series))
    fihi = np.sum(_fihi(tc, m, w, time_series))
    gihi = np.sum(_gihi(tc, m, w, time_series))
    
    yi = np.sum(_yi(price_series))
    yifi = np.sum(_yifi(tc, m, time_series, price_series))
    yigi = np.sum(_yigi(tc, m, w, time_series, price_series))
    yihi = np.sum(_yihi(tc, m, w, time_series, price_series))
    
    matrix_1 = np.matrix([
        [N,  fi,       gi,       hi      ],
        [fi, fi_pow_2, figi,     fihi    ],
        [gi, figi,     gi_pow_2, gihi    ],
        [hi, fihi,     gihi,     hi_pow_2]
    ])
    
    matrix_2 = np.matrix([
        [yi],
        [yifi],
        [yigi],
        [yihi]
    ])
    
    product = np.linalg.solve(matrix_1, matrix_2)
    
    return [i[0] for i in product.tolist()]

In [12]:
def compute_ds_lppls_confidence(n):    
    tLen = 180-(n*5)
    tradings_days_data = data.tail(tLen)
    time = np.linspace(0, tLen-1, tLen)
    price = [tradings_days_data["BAC"][i] for i in range(len(tradings_days_data["BAC"]))]
    data_series = [time, price]

    found_solution = False

    while not found_solution:

        # set limits for non-linear params
        limits = (
#             [tLen-(tLen*0.2), tLen+(tLen*0.2)],    # Critical Time + or - .2
            [-0.2, 0.2],
            [0.1, 0.9],                            # m : 0.1 ≤ m ≤ 0.9
            [6, 13],                               # ω : 6 ≤ ω ≤ 13     
        )

        # randomly choose vals for non-linear params 
        non_lin_vals = [random.uniform(a[0], a[1]) for a in limits]

        tc = non_lin_vals[0]
        m  = non_lin_vals[1] 
        w  = non_lin_vals[2]

        # params to pass to scipy.optimize
        seed = [tc, m, w]

        # scipy optimize minimize
        try:
            cofs = minimize(fun=func_restricted, x0=seed, args=(data_series), method='Nelder-Mead')

            if cofs.success:
                # determine if it falls in range:

                tc = cofs.x[0]
                m =  cofs.x[1]
                w =  cofs.x[2]

                tc_in_range = -0.05 < tc < 0.1
                m_in_range  = 0.01 < m < 1.2
                w_in_range  = 2 < w < 25
                n_oscillation = ((w/2)*np.log(abs((tLen - tc)/(tLen)))) > 2.5
#                 print((w/2)*np.log(abs((tLen - tc)/(tLen))))
#                 print(n_oscillation)
#                 damping = ""
#                 relative_error = ""

                if (tc_in_range and m_in_range and w_in_range and n_oscillation):
                    ds_lppls_confidence = True
                else: 
                    ds_lppls_confidence = False

                return {
                    'ds_lppls_confidence': ds_lppls_confidence,
                    'cof': cofs.x,
                }
        except Exception as e:
            print(e)
    
def main(time_range):
    solutions = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for foo in executor.map(compute_ds_lppls_confidence, time_range):
            solutions.append(foo)
    true_count = 0
    total_count = len(solutions)
    print(solutions)
    for i in solutions:
        if i["ds_lppls_confidence"] == True:
            true_count = true_count + 1
    return (true_count/total_count) if (true_count/total_count) != 0.0 else (true_count/total_count)*-1

In [13]:
ds_lppls = []

for i in range(1078):
    TIME_RANGE = [i for i in range(31)]
    data = pd.read_csv("../data/backtest.csv", index_col="date")
    # print(data.tail())
    data.drop(data.tail(i).index,inplace=True)
    # print(data.tail())
    start = time.time()
    ds_lppl = main(TIME_RANGE)
    end = time.time()
    ds_lppls.append(ds_lppl)
    print(end - start)
    print(ds_lppls)



/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarnin

/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in log
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in power
  after removing the cwd from sys.path.
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/Users/niejo05/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/niejo05/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: R

Singular matrix


/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: overflow encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: overflow encountered in multiply
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: overflow encountered in multiply
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: overflow encountered in multiply
/Users/niejo05/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[{'ds_lppls_confidence': False, 'cof': array([-2.77369017e-08,  5.16565288e-01,  3.69321910e+00])}, {'ds_lppls_confidence': False, 'cof': array([-5.24501974e-08,  2.75412474e+00,  1.69045813e+01])}, {'ds_lppls_confidence': False, 'cof': array([-1.94303133e-07,  2.61054118e+00,  1.62800667e+01])}, {'ds_lppls_confidence': False, 'cof': array([-9.14386562e-08,  2.42855750e+00,  1.56270821e+01])}, {'ds_lppls_confidence': False, 'cof': array([-1.27408112e-04,  2.18891401e+00,  1.49413984e+01])}, {'ds_lppls_confidence': False, 'cof': array([-7.19653911e-02,  1.93204287e+00, -1.15231707e-03])}, {'ds_lppls_confidence': False, 'cof': array([-2.50515735e-04,  1.62757590e+00,  1.36532905e+01])}, {'ds_lppls_confidence': False, 'cof': array([-0.01569397,  1.38337397, 13.0704016 ])}, {'ds_lppls_confidence': False, 'cof': array([-4.61206734e-07,  1.26362494e+00,  1.23334078e+01])}, {'ds_lppls_confidence': False, 'cof': array([-1.42110872e-07,  1.19923355e+00,  1.17318575e+01])}, {'ds_lppls_confidence

...

Singular matrix
[{'ds_lppls_confidence': False, 'cof': array([-0.19287426,  1.18506265,  9.8389058 ])}, {'ds_lppls_confidence': False, 'cof': array([-4.95052416e-03,  8.17510119e-01,  9.46706062e+00])}, {'ds_lppls_confidence': False, 'cof': array([-0.04213794,  0.58962267,  8.75849165])}, {'ds_lppls_confidence': False, 'cof': array([-0.13490315,  0.59639923,  8.07569999])}, {'ds_lppls_confidence': False, 'cof': array([-0.1445649 ,  0.8913163 ,  7.60636203])}, {'ds_lppls_confidence': False, 'cof': array([-0.3923604 ,  0.57552664,  6.93520464])}, {'ds_lppls_confidence': False, 'cof': array([-0.01651594,  0.57311586,  6.25481871])}, {'ds_lppls_confidence': False, 'cof': array([-0.11643436,  0.66004548,  8.17481788])}, {'ds_lppls_confidence': False, 'cof': array([-0.32483283,  0.58604656,  7.49165676])}, {'ds_lppls_confidence': False, 'cof': array([-0.06994773,  0.97575749,  6.74526652])}, {'ds_lppls_confidence': False, 'cof': array([-0.04161772,  1.03183186,  6.45425407])}, {'ds_lppls_con

/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in power
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarn

/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in power
  after removing the cwd from sys.path.
/Users/niejo05/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/Users/niejo05/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/niejo05/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/niejo05/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/niejo05/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83:

KeyboardInterrupt: 

Process ForkProcess-870:
Process ForkProcess-871:
Process ForkProcess-872:
Process ForkProcess-868:
Process ForkProcess-869:
Process ForkProcess-865:
Process ForkProcess-867:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/niejo05/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/niejo05/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/niejo05/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/niejo05/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/niejo05/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  Fil

In [ ]:
ds_lppl_confidence = pd.DataFrame([0.2903225806451613, 0.25806451612903225, 0.2903225806451613, 0.3548387096774194, 0.22580645161290322, 0.12903225806451613, 0.41935483870967744, 0.3548387096774194, 0.41935483870967744, 0.25806451612903225, 0.03225806451612903, 0.0967741935483871, 0.0967741935483871, 0.06451612903225806, 0.0967741935483871, 0.0967741935483871, 0.12903225806451613, 0.16129032258064516, 0.06451612903225806, 0.25806451612903225, 0.0967741935483871, 0.16129032258064516, 0.1935483870967742, 0.16129032258064516, 0.16129032258064516, 0.2903225806451613, 0.1935483870967742, 0.3548387096774194, 0.2903225806451613, 0.3870967741935484, 0.41935483870967744, 0.3870967741935484, 0.45161290322580644, 0.3870967741935484, 0.5483870967741935, 0.5161290322580645, 0.5483870967741935, 0.5483870967741935, 0.5806451612903226, 0.7096774193548387, 0.5806451612903226, 0.6129032258064516, 0.7419354838709677, 0.6774193548387096, 0.7419354838709677, 0.6129032258064516, 0.4838709677419355, 0.6129032258064516, 0.3870967741935484, 0.4838709677419355, 0.3225806451612903, 0.4838709677419355, 0.41935483870967744, 0.5483870967741935, 0.5161290322580645, 0.7096774193548387, 0.8064516129032258, 0.7741935483870968, 0.7741935483870968, 0.5161290322580645, 0.7741935483870968, 0.9032258064516129, 0.7419354838709677, 0.7096774193548387, 0.41935483870967744, 0.41935483870967744, 0.1935483870967742, 0.2903225806451613, 0.3225806451612903, 0.3548387096774194, 0.2903225806451613, 0.22580645161290322, 0.22580645161290322, 0.22580645161290322, 0.2903225806451613, 0.1935483870967742, 0.0967741935483871, 0.22580645161290322, 0.3548387096774194, 0.2903225806451613, 0.25806451612903225, 0.1935483870967742, 0.22580645161290322, 0.22580645161290322, 0.12903225806451613, 0.1935483870967742, 0.06451612903225806, 0.1935483870967742, 0.25806451612903225, 0.25806451612903225, 0.2903225806451613, 0.16129032258064516, 0.22580645161290322, 0.0967741935483871, 0.06451612903225806, 0.0967741935483871, 0.12903225806451613, 0.16129032258064516, 0.12903225806451613, 0.0967741935483871, 0.0967741935483871, 0.25806451612903225, 0.25806451612903225, 0.1935483870967742, 0.2903225806451613, 0.1935483870967742, 0.1935483870967742, 0.25806451612903225, 0.22580645161290322, 0.3548387096774194, 0.3870967741935484, 0.25806451612903225, 0.41935483870967744, 0.45161290322580644, 0.2903225806451613, 0.1935483870967742, 0.2903225806451613, 0.1935483870967742, 0.22580645161290322, 0.22580645161290322, 0.3225806451612903, 0.3225806451612903, 0.16129032258064516, 0.22580645161290322, 0.0967741935483871, 0.0967741935483871, 0.06451612903225806, 0.03225806451612903, 0.0, 0.06451612903225806, 0.12903225806451613, 0.22580645161290322, 0.06451612903225806, 0.22580645161290322, 0.12903225806451613, 0.12903225806451613, 0.16129032258064516, 0.0967741935483871, 0.16129032258064516, 0.16129032258064516, 0.0967741935483871, 0.03225806451612903, 0.12903225806451613, 0.12903225806451613, 0.1935483870967742, 0.1935483870967742, 0.25806451612903225, 0.3870967741935484, 0.3870967741935484, 0.5161290322580645, 0.6129032258064516, 0.4838709677419355, 0.6451612903225806, 0.7419354838709677, 0.7741935483870968, 0.5483870967741935, 0.7096774193548387, 0.5806451612903226, 0.5161290322580645, 0.5161290322580645, 0.6129032258064516, 0.45161290322580644, 0.5483870967741935, 0.45161290322580644, 0.5483870967741935, 0.4838709677419355, 0.25806451612903225, 0.41935483870967744, 0.3548387096774194, 0.3548387096774194, 0.2903225806451613, 0.2903225806451613, 0.3548387096774194, 0.22580645161290322, 0.3870967741935484, 0.3225806451612903, 0.25806451612903225, 0.3870967741935484, 0.3548387096774194, 0.2903225806451613, 0.25806451612903225, 0.3225806451612903, 0.41935483870967744, 0.3870967741935484, 0.3870967741935484, 0.3870967741935484, 0.3870967741935484, 0.3225806451612903, 0.1935483870967742, 0.1935483870967742, 0.06451612903225806, 0.0967741935483871, 0.0967741935483871, 0.12903225806451613, 0.25806451612903225, 0.3548387096774194, 0.1935483870967742, 0.2903225806451613, 0.3548387096774194, 0.3548387096774194])

In [ ]:
ds_lppl_confidence = ds_lppl_confidence.reindex(index=ds_lppl_confidence.index[::-1])

In [ ]:
ds_lppl_confidence.to_csv('/Users/niejo05/Desktop/lppls_confidence.csv')